In [ ]:
#config
import fiftyone as fo
import os


file_path = "/tf/testing"
#print(planes)
batch_size = 1000

dataset = fo.load_dataset("plane-dataset")


In [ ]:
session = fo.launch_app(dataset, auto=False)

In [ ]:
view = dataset.exists("planebox").skip(12000).limit(1000).match({"tags": {"$nin": ["relabel"]}})#.match({"relabel": {"$exists": False, "$eq": None}})
print(view)
session.view = view

In [ ]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.append("relabel")
    sample.save() 

#### Find the number of photos targeted for relabeling

In [ ]:
relabel_view = dataset.match_tags("relabel")
print(relabel_view)

### Find the number of images with good bounding boxes
Add a tag that bounding boxes are good

In [ ]:
good_view = dataset.exists("planebox").match({"tags": {"$nin": ["relabel"]}})
print(good_view)
#session.view = good_view

for sample in good_view:
    sample.tags.append("good_box")
    sample.save() 

In [ ]:
good_box_view = dataset.match_tags("good_box")
print(good_box_view)


### Create Bounding Boxes with the Model
For the image where the bounding boxes have been confirmed, we will now create bounding boxes with the normalized model.
It will print out the Sample ID for samples that either do not have a normalize model or do not have a plane detected

In [ ]:
good_box_view = dataset.match_tags("good_box")
bad_detections=[]
bad_models=[]
for sample in good_box_view:
    #print(sample)
    planeDetections = sample["planebox"].detections

    if len(planeDetections)==0:
        print(sample["id"])
        bad_detections.append(sample["id"])
    else:    
        bbox = planeDetections[0]["bounding_box"]
        if sample["norm_model"] == None:
            print(sample["id"])
            bad_models.append(sample["id"])
        else:
            label = sample["norm_model"].label
            detections = [fo.Detection(label=label, bounding_box=bbox)]
            sample["modelbox"] = fo.Detections(detections=detections)
            sample.save()

We are going to find that some samples do not have a plane detected or no model. When there is no model found, remove the `good_box` tag. When there is no detection, remove `good_box` tag and add a `relabel` tag.

In [ ]:
for sample in dataset.select(bad_detections):
    sample.tags.remove("good_box")
    sample.tags.append("relabel")
    sample.save()
    
for sample in dataset.select(bad_models):
    sample.tags.remove("good_box")
    sample.save()

In [ ]:
good_box_view = dataset.match_tags("good_box")
print(good_box_view)
session.view = good_box_view